- ⁠Entendimento do negócio
- ⁠Compreensão dos dados
- ⁠Tratamento e Limpeza dos dados
- ⁠Analise extrair informações relevantes pra conseguir responder perguntas

In [105]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [106]:
dados = pd.read_csv('/Users/lucas/pos-tech/estudo de caso/estudo-de-caso/dados/Customer_support_data.csv')

1. Identificar os principais canais de atendimento utilizados pelos Clientes
2. Avaliar o desempenho dos analistas de suporte.
3. Analisar o tempo de resposta e resolução dos tickets.
4. Identificar os principais motivos de satisfação e insatisfação dos clientes

In [107]:
#Channel_name, category, Issue_reported at, Issue_responded, Customer_City, Product_category, Agent_name, Agent Shift,
#CSAT Score

 
dados_clean = dados[['channel_name', 'category', 'Issue_reported at', 'issue_responded', 'Customer_City', 'Product_category', 'Agent_name', 'Agent Shift', 'CSAT Score']]


In [108]:
columns = ['channel_name','category','issue_reported','issue_responded','custumer_city','product_category','agent_name', 'agent_shift','csat']
dados_clean.columns = columns
dados_clean.head()

,channel_name,category,issue_reported,issue_responded,custumer_city,product_category,agent_name,agent_shift,csat
0,Outcall,Product Queries,01/08/2023 11:13,01/08/2023 11:47,NaN,NaN,Richard Buchanan,Morning,5
1,Outcall,Product Queries,01/08/2023 12:52,01/08/2023 12:54,NaN,NaN,Vicki Collins,Morning,5
2,Inbound,Order Related,01/08/2023 20:16,01/08/2023 20:38,NaN,NaN,Duane Norman,Evening,5
3,Inbound,Returns,01/08/2023 20:56,01/08/2023 21:16,NaN,NaN,Patrick Flores,Evening,5
4,Inbound,Cancellation,01/08/2023 10:30,01/08/2023 10:32,NaN,NaN,Christopher Sanchez,Morning,5


In [109]:
dados_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   channel_name      85907 non-null  object
 1   category          85907 non-null  object
 2   issue_reported    85907 non-null  object
 3   issue_responded   85907 non-null  object
 4   custumer_city     17079 non-null  object
 5   product_category  17196 non-null  object
 6   agent_name        85907 non-null  object
 7   agent_shift       85907 non-null  object
 8   csat              85907 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 5.9+ MB


In [110]:
dados_clean.drop(columns=['custumer_city', 'product_category'], inplace=True)

In [111]:
dados_clean.head()

,channel_name,category,issue_reported,issue_responded,agent_name,agent_shift,csat
0,Outcall,Product Queries,01/08/2023 11:13,01/08/2023 11:47,Richard Buchanan,Morning,5
1,Outcall,Product Queries,01/08/2023 12:52,01/08/2023 12:54,Vicki Collins,Morning,5
2,Inbound,Order Related,01/08/2023 20:16,01/08/2023 20:38,Duane Norman,Evening,5
3,Inbound,Returns,01/08/2023 20:56,01/08/2023 21:16,Patrick Flores,Evening,5
4,Inbound,Cancellation,01/08/2023 10:30,01/08/2023 10:32,Christopher Sanchez,Morning,5


In [112]:
# mudando o tipo das datas em datetime 
#dados_clean[['issue_reported', 'issue_responded']] = dados_clean[['issue_reported', 'issue_responded']].astype('datetime64[ns]')
dados_clean['issue_reported'] = pd.to_datetime(dados_clean['issue_reported'], format='%d/%m/%Y %H:%M')
dados_clean['issue_responded'] = pd.to_datetime(dados_clean['issue_responded'], format='%d/%m/%Y %H:%M')

In [113]:
dados_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   channel_name     85907 non-null  object        
 1   category         85907 non-null  object        
 2   issue_reported   85907 non-null  datetime64[ns]
 3   issue_responded  85907 non-null  datetime64[ns]
 4   agent_name       85907 non-null  object        
 5   agent_shift      85907 non-null  object        
 6   csat             85907 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 4.6+ MB


In [114]:
dados_clean.head()

,channel_name,category,issue_reported,issue_responded,agent_name,agent_shift,csat
0,Outcall,Product Queries,2023-08-01 11:13:00,2023-08-01 11:47:00,Richard Buchanan,Morning,5
1,Outcall,Product Queries,2023-08-01 12:52:00,2023-08-01 12:54:00,Vicki Collins,Morning,5
2,Inbound,Order Related,2023-08-01 20:16:00,2023-08-01 20:38:00,Duane Norman,Evening,5
3,Inbound,Returns,2023-08-01 20:56:00,2023-08-01 21:16:00,Patrick Flores,Evening,5
4,Inbound,Cancellation,2023-08-01 10:30:00,2023-08-01 10:32:00,Christopher Sanchez,Morning,5


# 1. Identificar os principais canais de atendimento utilizados pelos Clientes


In [115]:
dados_clean['channel_name'].groupby(dados_clean['channel_name']).count()

channel_name
Email       3023
Inbound    68142
Outcall    14742
Name: channel_name, dtype: int64

Resposta: Com base nos três atendimentos o que mais recebe atendimento é o Inbound, que seria quando o cliente faz uma ligação direta para o suporte

# 4. Identificar os principais motivos de satisfação e insatisfação dos clientes

CSAT:

1-3: negativo

4-5: positivo

In [116]:
dados_clean['avaliacao_numerica'] = dados_clean['csat'].map(lambda x: 0 if x <= 3 else 1)


In [117]:
dados_clean['avaliacao_numerica'].describe()

count    85907.000000
mean         0.824566
std          0.380340
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: avaliacao_numerica, dtype: float64

In [118]:
frequencia_avalaicao = dados_clean['avaliacao_numerica'].groupby(dados_clean['category']).count().sort_values(ascending=False) 
frequencia_avalaicao

category
Returns               44097
Order Related         23215
Refund Related         4550
Product Queries        3692
Shopzilla Related      2792
Payments related       2327
Feedback               2294
Cancellation           2212
Offers & Cashback       480
Others                   99
App/website              84
Onboarding related       65
Name: avaliacao_numerica, dtype: int64

In [119]:
# quais categorias tiveram avaliacao_numerica 1
avaliacao_positiva = dados_clean['avaliacao_numerica'].groupby(dados_clean['category']).sum().sort_values(ascending=False)

((avaliacao_positiva/frequencia_avalaicao)*100).round(2).sort_values(ascending=False)


category
App/website           86.90
Payments related      85.30
Returns               85.24
Shopzilla Related     83.99
Offers & Cashback     82.92
Refund Related        82.33
Onboarding related    81.54
Feedback              79.86
Order Related         78.56
Product Queries       76.90
Cancellation          75.90
Others                59.60
Name: avaliacao_numerica, dtype: float64

In [120]:
avaliacao_negativa = dados_clean[['category', 'avaliacao_numerica']].query('avaliacao_numerica == 0').groupby('category').count().sort_values('avaliacao_numerica', ascending=False)

In [121]:
avaliacao_negativa

,avaliacao_numerica
category,
Returns,6507
Order Related,4978
Product Queries,853
Refund Related,804
Cancellation,533
Feedback,462
Shopzilla Related,447
Payments related,342
Offers & Cashback,82


In [122]:
avaliacao_negativa['avaliacao_numerica_relativa'] = (avaliacao_negativa['avaliacao_numerica']/frequencia_avalaicao)*100

In [126]:
avaliacao_negativa.sort_values('avaliacao_numerica_relativa', ascending=False).round(2)

,avaliacao_numerica,avaliacao_numerica_relativa
category,,
Others,40,40.40
Cancellation,533,24.10
Product Queries,853,23.10
Order Related,4978,21.44
Feedback,462,20.14
Onboarding related,12,18.46
Refund Related,804,17.67
Offers & Cashback,82,17.08
Shopzilla Related,447,16.01


In [ ]:
# Os principais motivos de satisfação dos clientes são: App/website, payments related e retorns
# Os principais motivos de insatisfação dos clientes são: Cacellation, product queries e Order Related 

# 3. Analisar o tempo de resposta e resolução dos tickets.
